In [1]:
import requests
import time
import numpy as np
import pandas as pd
import requests
import datetime
import tensorflow as tf
import pickle
import os

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

In [3]:
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.cron import CronTrigger

In [4]:
headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.8",
    "Origin": "https://www.moneycontrol.com",
    "Referer": "https://www.moneycontrol.com/",
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Mobile Safari/537.36",
}

In [5]:
def calculate_length_and_sign(data1):
    length = data1["c"] - data1["o"]
    sign = length.apply(lambda x: 0 if x < 0 else 1)
    data1["Length"] = length
    data1["Sign"] = sign
    return data1

In [6]:
def normalize_columnwise(
    df, cols, min_max_scaling=False, standard_scaling=False, log_normalization=False
):

    for col in cols:
        if min_max_scaling:

            df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
        elif standard_scaling:

            df[col] = (df[col] - df[col].mean()) / df[col].std()
        elif log_normalization:

            df[col] = np.log(df[col])
        else:
            raise ValueError("Please specify a normalization technique.")

    return df

In [24]:
def prepare_data():
    data1 = pd.read_csv("training.csv")
    data1.set_index("time", inplace=True)
    data1["Open_Change"] = data1["o"].pct_change() * 100
    data1["High_Change"] = data1["h"].pct_change() * 100
    data1["Low_Change"] = data1["l"].pct_change() * 100
    data1["Close_Change"] = data1["c"].pct_change() * 100
    data1 = calculate_length_and_sign(data1)
    data1.dropna(inplace=True)
    df_normalized = normalize_columnwise(
        data1,
        ["c", "o", "h", "l"],
        min_max_scaling=True,
        standard_scaling=False,
        log_normalization=False,
    )
    df_normalized = normalize_columnwise(
        data1,
        ["Open_Change", "High_Change", "Low_Change", "Close_Change", "Length"],
        min_max_scaling=False,
        standard_scaling=True,
        log_normalization=False,
    )
    df_normalized.to_csv("normalized_data.csv")
    return

In [25]:
prepare_data()

In [8]:
def predict_close():
    df_normalized = pd.read_csv("normalized_data.csv")
    df_normalized.set_index("time", inplace=True)
    df_normalized.drop("t", axis=1, inplace=True)
    X_train_close = df_normalized.iloc[: int(len(df_normalized) * 0.85)]
    X_val_close = df_normalized.iloc[int(len(df_normalized) * 0.85) : -1]
    X_test_close = df_normalized[-1:]
    Y_train_close = X_train_close["c"]
    Y_val_close = X_val_close["c"]
    Y_test_close = X_test_close["c"]
    X_train_close.drop("c", axis=1, inplace=True)
    X_val_close.drop("c", axis=1, inplace=True)
    X_test_close.drop("c", axis=1, inplace=True)

    model = Sequential(
        [
            layers.Input((14, 1)),
            # Bidirectional LSTMs for extracting both forward and backward context
            layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
            layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
            # Standard LSTM for further processing
            layers.LSTM(32, dropout=0.5),
            # Dense layers for final feature extraction and prediction
            layers.Dense(32, activation="relu"),  # ReLU activation for non-linearity
            layers.Dense(16, activation="relu"),
            layers.Dense(1, activation="sigmoid"),  # Sigmoid for binary output
        ]
    )

    model.compile(
        loss="mse", optimizer=Adam(learning_rate=0.001), metrics=["mean_absolute_error"]
    )
    ealy_stopping = EarlyStopping(patience=10, monitor="val_loss")

    model.fit(
        X_train_close,
        Y_train_close,
        epochs=100,
        validation_data=(X_val_close, Y_val_close),
        callbacks=[ealy_stopping],
    )

    # Save the model using pickle
    # with open('model_close.pkl', 'wb') as f:
    #     pickle.dump(model, f)
    model.save("model_close.h5")

    y_pred = model.predict(X_test_close)
    # print(y_pred)

    df = pd.read_csv("training.csv")
    df.set_index("time", inplace=True)
    original_val_close = (y_pred - df_normalized["c"].min()) / (
        df_normalized["c"].max() - df_normalized["c"].min()
    ) * (df["c"].max() - df["c"].min()) + df["c"].min()
    # print(original_val_close)
    return original_val_close

In [9]:
def predict_high():
    df_normalized = pd.read_csv("normalized_data.csv")
    df_normalized.set_index("time", inplace=True)
    df_normalized.drop("t", axis=1, inplace=True)
    X_train_high = df_normalized.iloc[: int(len(df_normalized) * 0.85)]
    X_val_high = df_normalized.iloc[int(len(df_normalized) * 0.85) : -1]
    X_test_high = df_normalized[-1:]
    Y_train_high = X_train_high["h"]
    Y_val_high = X_val_high["h"]
    Y_test_high = X_test_high["h"]
    X_train_high.drop("h", axis=1, inplace=True)
    X_val_high.drop("h", axis=1, inplace=True)
    X_test_high.drop("h", axis=1, inplace=True)

    model = Sequential(
        [
            layers.Input((14, 1)),
            # Bidirectional LSTMs for extracting both forward and backward context
            layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
            layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
            # Standard LSTM for further processing
            layers.LSTM(32, dropout=0.5),
            # Dense layers for final feature extraction and prediction
            layers.Dense(32, activation="relu"),  # ReLU activation for non-linearity
            layers.Dense(16, activation="relu"),
            layers.Dense(1, activation="sigmoid"),  # Sigmoid for binary output
        ]
    )

    model.compile(
        loss="mse", optimizer=Adam(learning_rate=0.001), metrics=["mean_absolute_error"]
    )
    ealy_stopping = EarlyStopping(patience=10, monitor="val_loss")

    model.fit(
        X_train_high,
        Y_train_high,
        epochs=100,
        validation_data=(X_val_high, Y_val_high),
        callbacks=[ealy_stopping],
    )

    # Save the model using pickle
    # with open('model_high.pkl', 'wb') as f:
    #     pickle.dump(model, f)
    model.save("model_high.h5")
    y_pred = model.predict(X_test_high)
    df = pd.read_csv("training.csv")
    df.set_index("time", inplace=True)
    original_val_high = (y_pred - df_normalized["h"].min()) / (
        df_normalized["h"].max() - df_normalized["h"].min()
    ) * (df["h"].max() - df["h"].min()) + df["h"].min()
    return original_val_high

In [10]:
def predict_low():
    df_normalized = pd.read_csv("normalized_data.csv")
    df_normalized.set_index("time", inplace=True)
    df_normalized.drop("t", axis=1, inplace=True)
    X_train_low = df_normalized.iloc[: int(len(df_normalized) * 0.85)]
    X_val_low = df_normalized.iloc[int(len(df_normalized) * 0.85) : -1]
    X_test_low = df_normalized[-1:]
    Y_train_low = X_train_low["l"]
    Y_val_low = X_val_low["l"]
    Y_test_low = X_test_low["l"]
    X_train_low.drop("l", axis=1, inplace=True)
    X_val_low.drop("l", axis=1, inplace=True)
    X_test_low.drop("l", axis=1, inplace=True)

    model = Sequential(
        [
            layers.Input((14, 1)),
            # Bidirectional LSTMs for extracting both forward and backward context
            layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
            layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
            # Standard LSTM for further processing
            layers.LSTM(32, dropout=0.5),
            # Dense layers for final feature extraction and prediction
            layers.Dense(32, activation="relu"),  # ReLU activation for non-linearity
            layers.Dense(16, activation="relu"),
            layers.Dense(1, activation="sigmoid"),  # Sigmoid for binary output
        ]
    )

    model.compile(
        loss="mse", optimizer=Adam(learning_rate=0.001), metrics=["mean_absolute_error"]
    )
    ealy_stopping = EarlyStopping(patience=10, monitor="val_loss")

    model.fit(
        X_train_low,
        Y_train_low,
        epochs=100,
        validation_data=(X_val_low, Y_val_low),
        callbacks=[ealy_stopping],
    )

    # Save the model using pickle
    print(type(model))
    # with open('model_low.pkl', 'wb') as f:
    #     pickle.dump(model, f)
    model.save("model_low.h5")
    y_pred = model.predict(X_test_low)
    df = pd.read_csv("training.csv")
    df.set_index("time", inplace=True)
    original_val_low = (y_pred - df_normalized["l"].min()) / (
        df_normalized["l"].max() - df_normalized["l"].min()
    ) * (df["l"].max() - df["l"].min()) + df["l"].min()
    return original_val_low

In [11]:


def update_csv1(timestamp, original_val_high, original_val_low, original_val_close,open_val):
  if os.path.isfile("frontend_data.csv") and os.path.getsize("frontend_data.csv") > 30:
   
    df = pd.read_csv("frontend_data.csv")
    print(df.index[-1])
    # df.set_index("time", inplace=True)
    if int(df['time'][-1:]) == timestamp:
        # print('hi')
        # remove one row from df
        df.drop(df.index[-1], inplace=True)
        # print('removed ')
        print(df)
        n = pd.DataFrame(
            {
                "time": [timestamp],
                "open": [open_val],
                "high": [original_val_high],
                "low": [original_val_low],
                "close": [original_val_close],
            }
        )
        print(n)
        df = pd.concat([df, n])
        df.set_index("time", inplace=True)
        # print(df)
        df.to_csv("frontend_data.csv")
        # print(df)

        # df.loc[timestamp] = [1,2,3,4]
    else:
        n = pd.DataFrame(
            {
                "time": [timestamp],
                "open": [open_val],
                "high": [original_val_high],
                "low": [original_val_low],
                "close": [original_val_close],
            }
        )
        df = pd.concat([df, n])
        df.set_index("time", inplace=True)
        df.to_csv("frontend_data.csv")
        # print(df)
    # print(df.index[-1]==timestamp)


  else:
    df = {
        "time": [timestamp],
        "open": [open_val],
        "high": [original_val_high],
        "low": [original_val_low],
        "close": [original_val_close],
    }
    df = pd.DataFrame(df)
    df.set_index("time", inplace=True)
    df.to_csv("frontend_data.csv")

In [12]:
def leftlive():
    if os.path.isfile("leftlive.csv"):
      df=pd.read_csv('today_data.csv')
      df["time"] = df["time"].astype(str)
      df["time"] = pd.to_datetime(df["time"]).apply(lambda x: int(x.timestamp()))
      df=df.drop(['t','hour','minute','year','day','month'], axis=1)
      df['open']=df['o']
      df['close']=df['c']
      df['high']=df['h']
      df['low']=df['l']
      df=df.drop(['o','h','l','c'],axis=1)
      df.set_index('time',inplace=True)
      df.to_csv('leftlive.csv')
    else:
       df = pd.DataFrame(columns=['time', 'open', 'close', 'high', 'low'])
       df.set_index('time', inplace=True)
       df.to_csv('leftlive.csv')
       

In [14]:
def update_csv(data, current_time):
    print(type(data))
    if os.path.isfile("today_data.csv") and os.path.getsize("today_data.csv") > 100:

        df = pd.read_csv("today_data.csv")
        df.set_index("time", inplace=True)

        timestamp = data.index[-1]
        if timestamp == df.index[-1]:
            df.loc[timestamp] = data.loc[timestamp]
        else:
            df.loc[timestamp] = data.loc[timestamp]

        df.to_csv("today_data.csv")
    else:
        df = pd.DataFrame(data)
        df.to_csv("today_data.csv")

In [15]:
def prepare_frontend():
  prepare_data()
       
  df = pd.read_csv("normalized_data.csv")
# check type of data in time column
  df["time"] = df["time"].astype(str)
  df["time"] = pd.to_datetime(df["time"]).apply(lambda x: int(x.timestamp()))
  timestamp = df["time"].iloc[-1]

  new_data_close = df[-1:]
  new_data_high = df[-1:]
  new_data_low = df[-1:]

# #  drop t anc c from new_data_close
  df_low = df.drop(["t", "time"], axis=1)
  df_high = df.drop(["t", "time"], axis=1)
  df_close = df.drop(["t", "time"], axis=1)
  new_data_close.drop(["t", "c", "time"], axis=1, inplace=True)
  new_data_high.drop(["t", "h", "time"], axis=1, inplace=True)
  new_data_low.drop(["t", "l", "time"], axis=1, inplace=True)
# print(new_data_close)
# load models
  model_close = load_model("model_close.h5")
  model_high = load_model("model_high.h5")
  model_low = load_model("model_low.h5")
# predict
  close = model_close.predict(new_data_close)
  high = model_high.predict(new_data_high)
  low = model_low.predict(new_data_low)
  close = close[0][0]
  high = high[0][0]
  low = low[0][0]
# close
  # print(close, high, low)
  data_pred = pd.read_csv("training.csv")
  data_pred.set_index("time", inplace=True)
  original_val_low = (low - df_low["l"].min()) / (
    df_low["l"].max() - df_low["l"].min()
) * (data_pred["l"].max() - data_pred["l"].min()) + data_pred["l"].min()

  original_val_close = (close - df_close["c"].min()) / (
    df_close["c"].max() - df_close["c"].min()
) * (data_pred["c"].max() - data_pred["c"].min()) + data_pred["c"].min()
  original_val_high = (high - df_high["h"].min()) / (
    df_high["h"].max() - df_high["h"].min()
) * (data_pred["h"].max() - data_pred["h"].min()) + data_pred["h"].min()
  print(original_val_low, original_val_close, original_val_high)

  open = data_pred["o"].iloc[-1]
  new_data = {
    "timestamp": [timestamp],
    "close": [original_val_close],
    "high": [original_val_high],
    "low": [original_val_low],
    "open": [open],
}
  new_df = pd.DataFrame(new_data)
  print(new_df)
  update_csv1(timestamp, original_val_high, original_val_low, original_val_close,open)


In [ ]:
prepare_frontend()

In [22]:
def create_csv():
    df = pd.read_csv("Banknifty_data.csv")
    data = pd.read_csv("today_data.csv")

    # merge the two dataframes
    df = pd.concat([df, data])
    df.set_index("time", inplace=True)

    # save it to training.csv
    df.to_csv("training.csv")

In [20]:


def fetch_data():
    current_time = datetime.datetime.now()
    today = int(datetime.datetime.now().timestamp())
    start_time = int(current_time.timestamp())
    countback = 500
    url = f"https://priceapi.moneycontrol.com//techCharts/indianMarket/index/history?symbol=in%3Bnbx&resolution=15&from={start_time}&to={today}&countback={countback}&currencyCode=INR"

    # Fetch the data from moneycontrol
    res = requests.get(url=url, headers=headers)
    df = pd.DataFrame(res.json())
    df["time"] = pd.to_datetime(df["t"], unit="s")
    df["t"] = pd.to_datetime(df["t"], unit="s")
    df["time"] = df["time"].dt.strftime("%Y-%m-%d %H:%M")
    data = df
    data.set_index("time", inplace=True)
    data.drop(["s", "v"], axis=1, inplace=True)
    data["hour"] = data["t"].dt.hour
    data["minute"] = data["t"].dt.minute
    data["month"] = data["t"].dt.month
    data["day"] = data["t"].dt.day
    data["year"] = data["t"].dt.year
    print(data)
    
    
    # yo csv
    # data.to_csv('Banknifty_data.csv')
    # update_csv(data, current_time)
    # create_csv()
    # prepare_frontend()
    # leftlive()
    return

In [ ]:
fetch_data()

In [18]:
def preict_candle():
    create_csv()
    prepare_data()
    predict_close()
    predict_high()
    predict_low()
    return

In [65]:
create_csv()

In [66]:
prepare_frontend()

C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_14260\647218034.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data_close.drop(["t", "c", "time"], axis=1, inplace=True)
C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_14260\647218034.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data_high.drop(["t", "h", "time"], axis=1, inplace=True)
C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_14260\647218034.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

1/1 [==============================] - 2s 2s/step
47607.13717589378 47681.08197126984 47624.5590865016
    timestamp         close          high           low     open
0  1712037600  47681.081971  47624.559087  47607.137176  47618.2
8


C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_14260\3783923540.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if int(df['time'][-1:]) == timestamp:


In [68]:
leftlive()

In [67]:
preict_candle()

C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_14260\2711691790.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_close.drop("c", axis=1, inplace=True)
C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_14260\2711691790.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_close.drop("c", axis=1, inplace=True)
C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_14260\2711691790.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

Epoch 1/100
13/13 [==============================] - 9s 152ms/step - loss: 0.0539 - mean_absolute_error: 0.1986 - val_loss: 0.0414 - val_mean_absolute_error: 0.1876
Epoch 2/100
13/13 [==============================] - 0s 29ms/step - loss: 0.0446 - mean_absolute_error: 0.1740 - val_loss: 0.0183 - val_mean_absolute_error: 0.1218
Epoch 3/100
13/13 [==============================] - 0s 32ms/step - loss: 0.0324 - mean_absolute_error: 0.1413 - val_loss: 0.0340 - val_mean_absolute_error: 0.1302
Epoch 4/100
13/13 [==============================] - 0s 30ms/step - loss: 0.0188 - mean_absolute_error: 0.1136 - val_loss: 0.0363 - val_mean_absolute_error: 0.1468
Epoch 5/100
13/13 [==============================] - 0s 33ms/step - loss: 0.0140 - mean_absolute_error: 0.1000 - val_loss: 0.0375 - val_mean_absolute_error: 0.1521
Epoch 6/100
13/13 [==============================] - 0s 31ms/step - loss: 0.0137 - mean_absolute_error: 0.0994 - val_loss: 0.0342 - val_mean_absolute_error: 0.1372
Epoch 7/100
13/

c:\Users\ATUL SINGH\Desktop\web\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 1s 1s/step


C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_14260\1707539254.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_high.drop("h", axis=1, inplace=True)
C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_14260\1707539254.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_high.drop("h", axis=1, inplace=True)
C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_14260\1707539254.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

...

1/1 [==============================] - 1s 1s/step


C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_14260\3578925205.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_low.drop("l", axis=1, inplace=True)
C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_14260\3578925205.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_low.drop("l", axis=1, inplace=True)
C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_14260\3578925205.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

Epoch 1/100
13/13 [==============================] - 9s 159ms/step - loss: 0.0534 - mean_absolute_error: 0.2003 - val_loss: 0.0303 - val_mean_absolute_error: 0.1599
Epoch 2/100
13/13 [==============================] - 0s 33ms/step - loss: 0.0514 - mean_absolute_error: 0.1947 - val_loss: 0.0231 - val_mean_absolute_error: 0.1427
Epoch 3/100
13/13 [==============================] - 0s 31ms/step - loss: 0.0406 - mean_absolute_error: 0.1675 - val_loss: 0.0154 - val_mean_absolute_error: 0.1006
Epoch 4/100
13/13 [==============================] - 0s 30ms/step - loss: 0.0249 - mean_absolute_error: 0.1286 - val_loss: 0.0231 - val_mean_absolute_error: 0.1170
Epoch 5/100
13/13 [==============================] - 0s 30ms/step - loss: 0.0154 - mean_absolute_error: 0.1030 - val_loss: 0.0339 - val_mean_absolute_error: 0.1364
Epoch 6/100
13/13 [==============================] - 0s 31ms/step - loss: 0.0128 - mean_absolute_error: 0.0967 - val_loss: 0.0340 - val_mean_absolute_error: 0.1414
Epoch 7/100
13/

c:\Users\ATUL SINGH\Desktop\web\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 1s 1s/step


In [19]:
def del_today_data():
    # delete the today_data.csv file
    os.remove("today_data.csv")
    os.remove("frontend_data.csv")

In [67]:
# Initialize the first scheduler
scheduler1 = BackgroundScheduler()
trigger1 = CronTrigger(
    day_of_week="mon-fri", hour="9-10", minute="16,20,25,30,35,40,45,50,51,55,58"
)
scheduler1.add_job(fetch_data, trigger1)

scheduler2 = BackgroundScheduler()
trigger2 = CronTrigger(
    day_of_week="mon-fri",
    hour="10-15",
    minute="1,5,10,15,20,25,30,35,40,41,45,50,55,0",
)
scheduler2.add_job(fetch_data, trigger2)

scheduler3 = BackgroundScheduler()
trigger3 = CronTrigger(day_of_week="mon-fri", hour="15-16", minute="0,5,10,15,20,25,30")
scheduler3.add_job(fetch_data, trigger3)

scheduler4 = BackgroundScheduler()
trigger4 = CronTrigger(day_of_week="mon-fri", hour="9-15", minute="7,20,33,48,55")
scheduler4.add_job(preict_candle, trigger4)

scheduler5 = BackgroundScheduler()
trigger5 = CronTrigger(day_of_week="mon-fri", hour="15", minute="1,16,31")
scheduler5.add_job(preict_candle,trigger5)

scheduler6 = BackgroundScheduler()
trigger6 = CronTrigger(day_of_week="mon-fri", hour="15", minute="32")
scheduler6.add_job(del_today_data, trigger6)

<Job (id=39b9f1fbe8ae4e6b954cbe55a71f729b name=del_today_data)>

In [ ]:
fetch_data()

In [ ]:
preict_candle()

In [71]:
scheduler2.start()

In [72]:
scheduler2.shutdown()

In [ ]:
scheduler4.start()

C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_29632\2711691790.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_close.drop("c", axis=1, inplace=True)
C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_29632\2711691790.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_close.drop("c", axis=1, inplace=True)
C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_29632\2711691790.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

Epoch 1/100
14/14 [==============================] - 20s 283ms/step - loss: 0.0532 - mean_absolute_error: 0.1989 - val_loss: 0.0340 - val_mean_absolute_error: 0.1674
Epoch 2/100
14/14 [==============================] - 0s 28ms/step - loss: 0.0510 - mean_absolute_error: 0.1935 - val_loss: 0.0219 - val_mean_absolute_error: 0.1383
Epoch 3/100
14/14 [==============================] - 1s 64ms/step - loss: 0.0424 - mean_absolute_error: 0.1688 - val_loss: 0.0141 - val_mean_absolute_error: 0.0892
Epoch 4/100
14/14 [==============================] - 1s 84ms/step - loss: 0.0225 - mean_absolute_error: 0.1206 - val_loss: 0.0308 - val_mean_absolute_error: 0.1364
Epoch 5/100
14/14 [==============================] - 1s 77ms/step - loss: 0.0158 - mean_absolute_error: 0.1054 - val_loss: 0.0357 - val_mean_absolute_error: 0.1389
Epoch 6/100
14/14 [==============================] - 1s 75ms/step - loss: 0.0147 - mean_absolute_error: 0.0987 - val_loss: 0.0302 - val_mean_absolute_error: 0.1221
Epoch 7/100
14

c:\Users\ATUL SINGH\Desktop\web\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 4s 4s/step


C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_29632\1707539254.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_high.drop("h", axis=1, inplace=True)
C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_29632\1707539254.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_high.drop("h", axis=1, inplace=True)
C:\Users\ATUL SINGH\AppData\Local\Temp\ipykernel_29632\1707539254.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

Epoch 1/100
14/14 [==============================] - 25s 417ms/step - loss: 0.0619 - mean_absolute_error: 0.2131 - val_loss: 0.0364 - val_mean_absolute_error: 0.1756
Epoch 2/100
14/14 [==============================] - 1s 51ms/step - loss: 0.0610 - mean_absolute_error: 0.2105 - val_loss: 0.0265 - val_mean_absolute_error: 0.1511
Epoch 3/100
14/14 [==============================] - 1s 50ms/step - loss: 0.0527 - mean_absolute_error: 0.1921 - val_loss: 0.0202 - val_mean_absolute_error: 0.1111
Epoch 4/100
14/14 [==============================] - 1s 58ms/step - loss: 0.0225 - mean_absolute_error: 0.1189 - val_loss: 0.0448 - val_mean_absolute_error: 0.1566
Epoch 5/100
14/14 [==============================] - 1s 65ms/step - loss: 0.0210 - mean_absolute_error: 0.1197 - val_loss: 0.0345 - val_mean_absolute_error: 0.1296
Epoch 6/100
14/14 [==============================] - 1s 83ms/step - loss: 0.0196 - mean_absolute_error: 0.1152 - val_loss: 0.0381 - val_mean_absolute_error: 0.1380
Epoch 7/100
14

Epoch 1/100
14/14 [==============================] - 19s 338ms/step - loss: 0.0513 - mean_absolute_error: 0.1937 - val_loss: 0.0304 - val_mean_absolute_error: 0.1583
Epoch 2/100
14/14 [==============================] - 1s 55ms/step - loss: 0.0470 - mean_absolute_error: 0.1840 - val_loss: 0.0198 - val_mean_absolute_error: 0.1263
Epoch 3/100
14/14 [==============================] - 1s 54ms/step - loss: 0.0333 - mean_absolute_error: 0.1436 - val_loss: 0.0299 - val_mean_absolute_error: 0.1298
Epoch 4/100
14/14 [==============================] - 1s 49ms/step - loss: 0.0194 - mean_absolute_error: 0.1117 - val_loss: 0.0338 - val_mean_absolute_error: 0.1329
Epoch 5/100
14/14 [==============================] - 1s 60ms/step - loss: 0.0137 - mean_absolute_error: 0.0984 - val_loss: 0.0361 - val_mean_absolute_error: 0.1383
Epoch 6/100
14/14 [==============================] - 1s 67ms/step - loss: 0.0137 - mean_absolute_error: 0.0966 - val_loss: 0.0330 - val_mean_absolute_error: 0.1314
Epoch 7/100
14

c:\Users\ATUL SINGH\Desktop\web\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 4s 4s/step


In [ ]:
scheduler4.shutdown()

In [26]:
scheduler1.start()
scheduler2.start()
scheduler3.start()
scheduler4.start()
scheduler5.start()

In [29]:
scheduler1.shutdown()
scheduler2.shutdown()
scheduler3.shutdown()
scheduler4.shutdown()
scheduler5.shutdown()